In [6]:
import os
import datetime
import time
import json
import pandas as pd
import geopandas as gpd
import numpy as np
import requests
import shutil

import rasterio
from rasterio import plot
from shapely.geometry import MultiPolygon, shape, Point
from shapely_geojson import dumps

from pathlib import Path
from pprint import pprint
from zipfile import ZipFile

from planet import api
from planet.api import filters

to use your own api key, copy paste the `api_key/planet_api_key.py.dist` file in the same folder and remove the `.dist` extention. You can then replace the string with your own key. only the .dist will be pushed to the dist git rep. 

In [3]:
from api_key.planet_api_key import *

In [ ]:
def save_thumb(metadata_df):
    """ From the metadata dataframe, save the thumbnail
    in the corresponding folder:
    
    Args:
        metadata_df (pd.DataFrame)
        
    Return:
        stores thumbnails in folder
    """
    session = requests.Session()
    session.auth = (PLANET_API_KEY, '')
    auth = session.auth
    
    for index, row in metadata_df.iterrows():
        url = row.thumbnail
        date = row.date
        item_type = row.item_type
        cloud_cover = row.cloud_cover
        id_ = row.id
        sample_id = row.sample_id
        
        thumb_name = f'it{item_type}_cc{cloud_cover}_y{date.year}m{date.month}_{id_}.jpg'
        
        thumb_path = os.path.join(os.getcwd(),'thumbs', 
                                  sample_id, 
                                  str(date.year))
        
        Path(thumb_path).mkdir(parents=True, exist_ok=True)

        r = requests.get(url, auth=auth, stream=True)
        if r.status_code == 200:
            with open(os.path.join(thumb_path, thumb_name), 'wb') as f:
                r.raw.decode_content = True
                shutil.copyfileobj(r.raw, f)

In [ ]:
def build_request(aoi_geom, start_date, stop_date, cloud_cover=100):
    """build a data api search request for PS imagery.
    
    Args:
        aoi_geom (geojson): 
        start_date (datetime.datetime)
        stop_date (datetime.datetime)
    
    Returns:
        Request
    """
    
    query = filters.and_filter(
        filters.geom_filter(aoi_geom),
        filters.range_filter('cloud_cover', lte=cloud_cover),
        filters.date_range('acquired', gt=start_date),
        filters.date_range('acquired', lt=stop_date)
    )
    
    # Skipping REScene because is not orthorrectified and 
    # cannot be clipped.
    
    return filters.build_search_request(query, [
        'PSScene3Band', 
        'PSScene4Band', 
        'PSOrthoTile',
        'REOrthoTile',])

In [ ]:
# search the data api
def search_data_api(request, client, limit=100000):
    """ Search items from a given request.
    
    """
    result = client.quick_search(request)
    
    # this returns a generator
    return result.items_iter(limit=limit)

In [ ]:
def get_items(id_name, request):
    """ Get items using the request with the given parameters
    
    Args:
        row (geopandas.DataFrame.row): 
            gpd.df.row with two columns: id(index) and geometry
        
        start_date:
        
    """
    
    items = list(search_data_api(request, client))
    
    return [id_name, items]

In [ ]:
def get_dataframe(items):
    
    items_metadata = [(f['properties']['acquired'],
                     f['id'], 
                     f['properties']['item_type'],
                     f['_links']['thumbnail'],
                     f['_permissions'],
                     f['geometry'],
                     f['properties']['cloud_cover'],
                     f
                    ) for f in items[1]]
    
    # Store into dataframe
    df = pd.DataFrame(items_metadata)
    df[0] = pd.to_datetime(df[0])
    df.columns=[
        'date', 
        'id', 
        'item_type', 
        'thumbnail', 
        'permissions', 
        'footprint', 
        'cloud_cover', 
        'metadata'
    ]
    df['sample_id'] = items[0]
    df.sort_values(by=['date'], inplace=True)
    df.reset_index()
    
    return df

In [ ]:
def add_cover_area(metadata_df, sample_df):
    
    for idx, row in metadata_df.iterrows():
        
        g1 = sample_df.at[row.sample_id, 'geometry'] # sample geometry
        g2 = shape(row.footprint) # footprint geometry
        metadata_df.at[idx, 'cover_perc'] = (g1.intersection(g2).area/g1.area)

In [ ]:
def build_order_from_metadata(metadata_df, samples_df, sample_id):
    
    filtered_df = metadata_df[metadata_df.sample_id==sample_id]
    
    items_by_type = [(item_type, filtered_df[filtered_df.item_type == item_type].id.to_list())
              for item_type in filtered_df.item_type.unique()]
    
    products_bundles = {
        
        # Is not possible to ask for analytic_dn in PSScene3Band, so the next option is visual
        'PSScene3Band': "analytic,visual",
        'PSScene4Band': "analytic_udm2,analytic_sr,analytic",
        'PSOrthoTile': "analytic_5b_udm2,analytic_5b,analytic_udm2,analytic",
        'REOrthoTile': "analytic",
    }

    products_order = [
        {
            "item_ids":v, 
            "item_type":k, 
            "product_bundle": products_bundles[k]
        } for k, v in items_by_type
    ]
    
    # clip to AOI
    aoi_geojson = json.loads(dumps(samples_df.at[sample_id, 'geometry']))
    tools = [{
        'clip': {
            'aoi': aoi_geojson
        }
    },]
    
    order_request = {
        'name': f'sample_{sample_id}',
        'products': products_order,
        'tools': tools,
        'delivery': {
            'single_archive': True,
            'archive_filename':'{{name}}_{{order_id}}.zip',
            'archive_type':'zip'
        },
            'notifications': {
                       'email': False
        },
    }
    return order_request

In [ ]:
def filter_dataframe_items(dataframe, item_type_score, months_score):
    """Filter and score each item according to the season and item_type
    
    Return:
        Dataframe with only one selected image per year.
        
    """
    # Create a copy to avoid mutate the initial df
    df = dataframe.copy()
    
    item_count_per_year = dict(df.groupby(df.date.dt.year).size())
    
    for k_year in item_count_per_year.keys():
        
        # Filter only years with more than one image
        if item_count_per_year[k_year] > 1:

            for idx, row in metadata_df.iterrows():
                
                month = row.date.month

                df.at[idx, 'season_score'] = months_score[month]
                df.at[idx, 'item_score'] = item_type_score[row['item_type']]

                cloud_cover = row['cloud_cover']

                if cloud_cover == 0:
                    df.at[idx, 'cloud_score'] = 10
                elif cloud_cover < 0.005:
                    df.at[idx, 'cloud_score'] = 5
                else:
                    df.at[idx, 'cloud_score'] = 0
    
    df['total_score'] = df.season_score + \
                        df.item_score + \
                        df.cloud_score
    
    df = df.sort_values(by=['total_score', 'date'], ascending=False)
    df['year'] = df.date.dt.year
    df = df.drop_duplicates(subset=['year'], keep='first')
    df = df.sort_values(by=['date'], ascending=False)
    
    return df

In [ ]:
def track_order(order_id, client, num_loops=50):
    count = 0
    while(count < num_loops):
        count += 1
        order_info = client.get_individual_order(order_id).get()
        state = order_info['state']
        print(state)
        success_states = ['success', 'partial']
        if state == 'failed':
            raise Exception(response)
        elif state in success_states:
            break
        
        time.sleep(10)

# 1. Search items
### Get the samples dataframe

From a geojson plots file, create a geo pandas dataframe to store the geometries and the id of each plot, it'll be used as a geometry filter and to calculate the % of area covered by the items.

In [9]:
#create a geoDataFrame object from a .txt file 
if os.path.isfile(FILENAME):
    df = pd.read_csv(FILENAME, sep=' ')
    
    #filter only the `nb_rows` first rows
    nb_rows = len(df)
    filter_df  = df[df.index.isin(range(nb_rows))]
    df = filter_df
    
    #create the geodataframe 
    pts = [Point(df.loc[i][FILE_LNG], df.loc[i][FILE_LAT]) for i in range(len(df))]
    sample_gdf = gpd.GeoDataFrame(data={'geometry': pts}, index=df[FILE_ID], crs="EPSG:4326")
    sample_gdf.index.names = ['id']
    sample_gdf = sample_gdf.to_crs('ESRI:54009')
    sample_gdf['geometry'] = sample_gdf['geometry'].buffer(BUFFER_SIZE, cap_style=3)
    sample_gdf = sample_gdf.to_crs("EPSG:4326")

sample_gdf

,geometry
id,
323,"POLYGON ((25.59552 5.34852, 25.59469 5.31611, ..."
705,"POLYGON ((25.56903 5.32201, 25.56820 5.28960, ..."
830,"POLYGON ((25.57165 5.31605, 25.57082 5.28364, ..."
951,"POLYGON ((25.55911 5.30871, 25.55828 5.27631, ..."
1717,"POLYGON ((25.72331 5.25047, 25.72249 5.21806, ..."
...,...
35938957,"POLYGON ((29.74311 -13.43112, 29.74561 -13.463..."
35938978,"POLYGON ((29.74538 -13.43200, 29.74788 -13.464..."
35939051,"POLYGON ((29.74051 -13.43306, 29.74302 -13.465..."


In [8]:
# Read the samples and select the first geometry for test purposes

#samples_gdf = gpd.read_file('shp/samples.geojson')
#samples_gdf.drop('lnd_s__', axis=1, inplace=True)
#samples_gdf.set_index('id', drop=True, inplace=True)
#samples_gdf

,geometry
id,
8848_21404,"POLYGON ((-2.12363 5.38402, -2.12363 5.38688, ..."
8650_21398,"POLYGON ((-2.73005 7.17476, -2.73005 7.17762, ..."
8704_21284,"POLYGON ((-1.49995 6.68617, -1.49995 6.68903, ..."
8726_21476,"POLYGON ((-3.15945 6.48755, -3.15945 6.49041, ..."
8698_21346,"POLYGON ((-2.08237 6.73998, -2.08237 6.74284, ..."


### Connect to client

In [ ]:
client = api.ClientV1(api_key=PLANET_API_KEY)

### Define filters

In [ ]:
# define test data for the filter
start_date = datetime.datetime(2009, 1, 1)
stop_date = datetime.datetime(2020, 12, 31)
cloud_cover_lte = 0.01

## 1.1 Get items for individual ids ((optional))
### Get items and metadata using filters

In [ ]:
# Define AOI, by selecting the first row of the samples geodataframe

row_number = 1
aoi_geometry = json.loads(dumps(samples_gdf.iloc[row_number].geometry))
sample_id = samples_gdf.iloc[row_number].name

In [ ]:
request = build_request(aoi_geometry, start_date, stop_date, cloud_cover_lte)
items = get_items(sample_id, request)

# Transform items into a pandas dataframe with useful columns
metadata_df = get_dataframe(items)

### Calculate percentage of covered area

Calculate the percentage of covered area from the sample area with the item footprint

In [ ]:
# Mutate metadata_df and add the percentage of cover area
add_cover_area(metadata_df, samples_gdf)

In [ ]:
# Remove items that are under 100% of covered area
metadata_df = metadata_df[metadata_df.cover_perc == 1]

### Score items


In [ ]:
# item_type_score
item_type_score = {
    'PSScene4Band':10, 
    'PSScene3Band':10, 
    'PSOrthoTile':10,
    'REOrthoTile':0,
    'SkySatScene':0,
}

# season score
months_score = {
    1: 7, 7:0,
    2: 7, 8:0,
    3: 5, 9:0,
    4: 0, 10:5,
    5: 0, 11:10,
    6: 0, 12:10,
}

In [ ]:
selected_items = filter_dataframe_items(metadata_df, item_type_score, months_score)

### ((Optional)): Export thumbnails
Create thumbnails from the selected items (dataframe) and store them into a structured folder

In [ ]:
save_thumb(all_df)

## 1.2 Get items for all plots and store into a big df
### Loop over all plots
Loop over all plots and get the items.

In [ ]:
# Create a list of dataframes 

df_list = []
for index, row in samples_gdf.iterrows():
    
    aoi_geometry = json.loads(dumps(row.geometry))
    sample_id = row.name
    
    request = build_request(aoi_geometry, start_date, stop_date, cloud_cover_lte)
    items = get_items(sample_id, request)
    
    # Transform items into a pandas dataframe with useful columns
    metadata_df = get_dataframe(items)
    
    selected_items = filter_dataframe_items(metadata_df, item_type_score, months_score)
    
    df_list.append(selected_items)
    
    del metadata_df

In [ ]:
# Concatenate dataframes from the df_list
all_df = pd.concat(df_list)

# 2. Order assets

In [ ]:
# To create the order we need a dataframe with 
# filtered items, and a samples_gdf where is the geometry to clip 
# each item.

orders = []
for idx, row in samples_gdf.iterrows():

    order = build_order_from_metadata(all_df, samples_gdf, sample_id=idx)
    orders.append(order)

In [ ]:
orders[1]

### Request order
<font color='red'>The following lines will start the order in the planet server, after the order is created and is running, there is no way to stop it.</font>

In [ ]:
# order_info = client.create_order(orders[1]).get()

In [ ]:
order_id = order_info['id']
order_id

In [ ]:
orders = client.get_orders().get()

In [ ]:
track_order(order_id, client)

# 3. Download

In [ ]:
api_models = client.download_order(order_id)
api_models[0].get_body().write() # Write the iamge file